In [9]:
import torch
import numpy as np
index = torch.from_numpy(np.array([[1,2,0], [2,0,1]])).type(torch.LongTensor)
index.shape

torch.Size([2, 3])

In [10]:
index = index[:,:,None]
index.shape

torch.Size([2, 3, 1])

In [11]:
index

tensor([[[1],
         [2],
         [0]],

        [[2],
         [0],
         [1]]])

In [14]:
index = index.expand(-1,-1,10)
index

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

        [[2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]])

In [15]:
index.shape 

torch.Size([2, 3, 10])

In [16]:
inputs = torch.rand(2,3,10)
inputs  # # 可以理解为 2个batch 每个batch 3句话， 每句话10个词  只不过不是索引值是连续值

tensor([[[0.8499, 0.5743, 0.1163, 0.7354, 0.7336, 0.0461, 0.5596, 0.5597,
          0.8331, 0.8325],
         [0.3856, 0.3141, 0.2987, 0.2221, 0.7227, 0.9536, 0.9431, 0.8833,
          0.6936, 0.0054],
         [0.4265, 0.8442, 0.0826, 0.3865, 0.4617, 0.3623, 0.0011, 0.3030,
          0.3444, 0.3968]],

        [[0.2399, 0.8165, 0.5930, 0.0286, 0.9473, 0.1417, 0.3795, 0.2192,
          0.5359, 0.6525],
         [0.2812, 0.9378, 0.3347, 0.7453, 0.1059, 0.8147, 0.4491, 0.3992,
          0.0209, 0.1799],
         [0.1993, 0.1519, 0.2089, 0.3881, 0.1018, 0.3271, 0.7906, 0.7492,
          0.9048, 0.2077]]])

In [17]:
torch.gather(inputs, 1, index)

tensor([[[0.3856, 0.3141, 0.2987, 0.2221, 0.7227, 0.9536, 0.9431, 0.8833,
          0.6936, 0.0054],
         [0.4265, 0.8442, 0.0826, 0.3865, 0.4617, 0.3623, 0.0011, 0.3030,
          0.3444, 0.3968],
         [0.8499, 0.5743, 0.1163, 0.7354, 0.7336, 0.0461, 0.5596, 0.5597,
          0.8331, 0.8325]],

        [[0.1993, 0.1519, 0.2089, 0.3881, 0.1018, 0.3271, 0.7906, 0.7492,
          0.9048, 0.2077],
         [0.2399, 0.8165, 0.5930, 0.0286, 0.9473, 0.1417, 0.3795, 0.2192,
          0.5359, 0.6525],
         [0.2812, 0.9378, 0.3347, 0.7453, 0.1059, 0.8147, 0.4491, 0.3992,
          0.0209, 0.1799]]])

# 过程
- 准备输入
- 初始化参数矩阵
- 获取key、query和value
- 计算softmax
- 给value乘上score
- 给value 加权求和获取output1
- 重复4-7 获取output2、3

In [1]:
import torch
x = [
    [1,0,1,0], # Input1
    [0,2,0,2], # Input1
    [1,1,1,1] # Input1
]
x = torch.tensor(x, dtype=torch.float32)
x

tensor([[1., 0., 1., 0.],
        [0., 2., 0., 2.],
        [1., 1., 1., 1.]])

In [3]:
w_key = [
  [0, 0, 1],
  [1, 1, 0],
  [0, 1, 0],
  [1, 1, 0]
]
w_query = [
  [1, 0, 1],
  [1, 0, 0],
  [0, 0, 1],
  [0, 1, 1]
]
w_value = [
  [0, 2, 0],
  [0, 3, 0],
  [1, 0, 3],
  [1, 1, 0]
]
w_key = torch.tensor(w_key, dtype=torch.float32)
w_query = torch.tensor(w_query, dtype=torch.float32)
w_value = torch.tensor(w_value, dtype=torch.float32)

print("Weights for key: \n", w_key)
print("Weights for query: \n", w_query)
print("Weights for value: \n", w_value)

Weights for key: 
 tensor([[0., 0., 1.],
        [1., 1., 0.],
        [0., 1., 0.],
        [1., 1., 0.]])
Weights for query: 
 tensor([[1., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 1.]])
Weights for value: 
 tensor([[0., 2., 0.],
        [0., 3., 0.],
        [1., 0., 3.],
        [1., 1., 0.]])


In [4]:

keys = x @ w_key
querys = x @ w_query
values = x @ w_value

print("Keys: \n", keys)
print("Querys: \n", querys)
print("Values: \n", values)

Keys: 
 tensor([[0., 1., 1.],
        [4., 4., 0.],
        [2., 3., 1.]])
Querys: 
 tensor([[1., 0., 2.],
        [2., 2., 2.],
        [2., 1., 3.]])
Values: 
 tensor([[1., 2., 3.],
        [2., 8., 0.],
        [2., 6., 3.]])


In [5]:
attn_scores = querys @ keys.T
attn_scores

tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])

In [6]:
from torch.nn.functional import softmax
attn_scores_softmax = softmax(attn_scores, dim=-1)
attn_scores_softmax

tensor([[6.3379e-02, 4.6831e-01, 4.6831e-01],
        [6.0337e-06, 9.8201e-01, 1.7986e-02],
        [2.9539e-04, 8.8054e-01, 1.1917e-01]])

In [7]:
# For readability, approximate the above as follows
attn_scores_softmax = [
  [0.0, 0.5, 0.5],
  [0.0, 1.0, 0.0],
  [0.0, 0.9, 0.1]
]
attn_scores_softmax = torch.tensor(attn_scores_softmax)
print(attn_scores_softmax)

tensor([[0.0000, 0.5000, 0.5000],
        [0.0000, 1.0000, 0.0000],
        [0.0000, 0.9000, 0.1000]])


In [8]:
weighted_values = values[:,None] * attn_scores_softmax.T[:,:,None]
print(weighted_values)

tensor([[[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[1.0000, 4.0000, 0.0000],
         [2.0000, 8.0000, 0.0000],
         [1.8000, 7.2000, 0.0000]],

        [[1.0000, 3.0000, 1.5000],
         [0.0000, 0.0000, 0.0000],
         [0.2000, 0.6000, 0.3000]]])


In [9]:
weighted_values = values[:,None] * attn_scores_softmax.T[:,:,None]
print(weighted_values)
outputs = weighted_values.sum(dim=0)
print(outputs)


tensor([[[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[1.0000, 4.0000, 0.0000],
         [2.0000, 8.0000, 0.0000],
         [1.8000, 7.2000, 0.0000]],

        [[1.0000, 3.0000, 1.5000],
         [0.0000, 0.0000, 0.0000],
         [0.2000, 0.6000, 0.3000]]])
tensor([[2.0000, 7.0000, 1.5000],
        [2.0000, 8.0000, 0.0000],
        [2.0000, 7.8000, 0.3000]])
